# Movie Gross & Sentiment
### Group 9 Final Project - DNSC 6211

Team Members:
* Kelly Berdelle
* Jason Houghton
* Yuebo Li
* Qinya Wang
* Gaoshuang Zhu

## Project Structure

* Problem Statement
* Data Sources
* Python Procedures
* R Shiny Visualizations
* Conclusions with Linear Regression Models

### Problem: 
Can the box office success of a movie indicate the future sentiment of that movie?

If future sentiment can be predicted, then movie studios will be able to use that information to forecast future earnings and use that information for long-term licensing deals.

### Hypothesis:
There will be a positive relationship between the financial success of a movie at time of release and future sentiment.

### Test:
To test our hypothesis, we will compile a list of the top 200 movies by gross, adjusted for inflation to 2016 dollars.  We will then conduct a sentiment analysis on twitter for each of these movies as a marker for current sentiment.  We will determine the relationship between box office success and future sentiment by comparing the ajusted gross income to the sentiment score.

## Step 1: Scrape the movie data

We found a list of the top 200 movies ranked by gross, adjusted for inflation to 2016 dollars, at the following website: http://www.boxofficemojo.com/alltime/adjusted.htm.

We scraped the information from this website into a data frame including columns for Ranking, Movie Title, Studio, Adjusted Gross, Unadjusted Gross, and Year.

In [1]:
#imports
from bs4 import BeautifulSoup
import pandas as pd
import urllib

In [2]:
#scrape website for movie gross data
def scrapeBoxOffice():
    """
    Function to scrape data from BoxOfficeMojo
    Inputs: None
    Output: None
    Returns data frame containing search results
    """
    pageResults = []
        
    # Generate url
    fullUrl = 'http://www.boxofficemojo.com/alltime/adjusted.htm'
    
    # Read results from url into Beautiful Soup
    try:
        soup = BeautifulSoup(urllib.request.urlopen(fullUrl).read(), 'lxml')
    except Exception as e:
        pass
        return []
        
    body = soup.find('div', {'id': 'body'})
    
    mainTable = body.findAll('table', recursive=False)[1].tr.td.table
    
    rows = mainTable.findAll('tr')
    firstRow = True
    
    for row in rows:
        if firstRow == False:
            
            pageResult = {
                'Ranking': '',
                'Movie Title': '',
                'Studio': '',
                'Adjusted Gross': '',
                'Unadjusted Gross': '',
                'Year': '',
                }
                
            cols = row.findAll('td')
            pageResult['Ranking'] = cols[0].font.get_text(strip=True)
            pageResult['Movie Title'] = cols[1].font.a.b.get_text(strip=True)
            pageResult['Studio'] = cols[2].font.a.get_text(strip=True)
            pageResult['Adjusted Gross'] = cols[3].font.b.get_text(strip=True)
            pageResult['Unadjusted Gross'] = cols[4].font.get_text(strip=True)
            pageResult['Year'] = cols[5].font.get_text(strip=True)
            if pageResult['Unadjusted Gross'] == '2016':
                pageResult['Year'] = '2016'
                pageResult['Unadjusted Gross'] = pageResult['Adjusted Gross']
            pageResults.append(pageResult)
        firstRow = False   
    df = pd.DataFrame(pageResults, columns=['Ranking', 'Movie Title', 'Studio', 'Adjusted Gross', 'Unadjusted Gross', 'Year'])
    df['Adjusted Gross'].replace(regex=True, inplace=True, to_replace=r'\D', value=r'')
    df['Unadjusted Gross'].replace(regex=True, inplace=True, to_replace=r'\D', value=r'')
    df['Year'].replace(regex=True, inplace=True, to_replace=r'\D', value=r'')
    
    return df

In [3]:
#call the function to scrape the gross movie data
results = scrapeBoxOffice()

## Step 2: Get a sentiment score for the top 200 highest grossing movies

We used tweepy to gather tweets containing the title of each movie in our data frame. Each tweet was cleaned, and a sentiment score was conducted.  A column was added to our data frame for Sentiment Score.  The resulting data frame was then written to a csv file.

In [ ]:
#imports
import tweepy
from textblob import TextBlob
import numpy as np
import time
import csv

In [ ]:
def getAuthData():
    """
    Input: None
    Returns: User's twitter authorization data
    """
    with open('authdata.csv', 'r') as f: # When running replace 'authdata.csv' with user auth data file.
        reader = csv.reader(f)
        your_list = list(reader)

    authdata = {}   
    for element in your_list:
        authdata[element[0]] = element[1]

    return authdata

In [ ]:
def getTweets(searchTerm):
    """
    Input: Term to be searched in Twitter (movie title)
    Returns: A string containing all of the resulting Tweets
    """
    authdata = getAuthData()

    CONSUMER_KEY = authdata['CONSUMER_KEY']
    CONSUMER_SECRET = authdata['CONSUMER_SECRET']
    OAUTH_TOKEN = authdata['OAUTH_TOKEN']
    OAUTH_TOKEN_SECRET = authdata['OAUTH_TOKEN_SECRET']
    
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

    api = tweepy.API(auth)
    
    public_tweets = tweepy.Cursor(api.search, q=searchTerm).items(100)

    results= []

    for tweet in public_tweets:
        results.append(tweet.text)
    return results

In [ ]:
def remove_punctuation(tweetString):
    """
    Input: String containing Tweets
    Returns: String containing Tweets without punctuation markings
    """
    punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    s_sans_punct = ""
    for letter in tweetString:
        if (letter not in punctuation) and (letter in "abcdefghijklmnopqrstuvwxyz "):
            s_sans_punct += letter
    return s_sans_punct

In [ ]:
def getLowerCaseText(status_texts):
    """
    Input: String containing Tweets
    Returns: String containing Tweets in all lower case letters
    """
    lowered_texts = []
    for texts in status_texts:
        try: 
            mytext = str(texts.lower())
            lowered_texts.append(mytext)
        except:
            pass
    return lowered_texts

In [ ]:
def getCleanedTweets(lowered_texts):
    """
    Input: String containing Tweets
    Returns: String containing cleaned Tweets
    """
    cleanedTweets = []
    for text in lowered_texts:
        wordlist = str(text).split()
        wordlist_nopun = [ str(remove_punctuation(for_a_word)) for for_a_word in wordlist ]
        cleanedTweets.append(wordlist_nopun)
    return cleanedTweets

In [ ]:
def score(CleanedTweet):
    """
    Input: String containing cleaned Tweets
    Returns: A formatted score for the string of cleaned Tweets
    """
    strCT = []
    for i in CleanedTweet:
        strCT.append(' '.join(i))
    score= []     
    for j in strCT:
        analysis = TextBlob(j)
        score.append(analysis.sentiment.polarity)
    FormatScore = []
    for x in score:
        x = 50*(x+1)
        FormatScore.append(x)
    return FormatScore

In [ ]:
def tweepySentiment(searchTerm):
    """
    Input: Term to be searched in Twitter (movie title)
    Returns: Sentiment score for the search term (movie title)
    """
    # Counter to enact a time delay in order to avoid searching for too many Tweets in a fixed amount of time.
    global tweetCount
    if tweetCount > 650:
        time.sleep(60*16)
        tweetCount = 0
    # Executes functions to get Tweets, clean them, and produce a sentiment score
    movieTweets = getTweets(searchTerm)
    tweetCount += len(movieTweets)
    movieTweetsLCT = getLowerCaseText(movieTweets)
    cleanedMovieTweets = getCleanedTweets(movieTweetsLCT)
    movieScore = score(cleanedMovieTweets)
    sentimentScore = np.mean(movieScore)
    
    return sentimentScore

In [ ]:
# Prepare to get tweets and sentiments
tweetCount = 0
# Create a new column in the data frame for sentiment score
results['Sentiment Score'] = ''

In [ ]:
# Gather Tweets, conduct sentiment anlaysis, and add results to the data frame for each movie in the data frame
for movie in results['Movie Title']:
    results['Sentiment Score'][results['Movie Title'] == movie] = tweepySentiment(movie)

In [ ]:
# Write dataframe to a csv file
results.to_csv('BoxOfficeSentiment.csv')